# Importing Libs

In [1]:
import pandas as pd
import numpy as np
import glob
import os
pd.set_option('display.max_rows', None)

# Importing dataset

Place all your "file.csv" files in a folder named files and then use its path as mentioned below

In [2]:
path = r"files" # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

df = frame.rename(columns={'IsDeleted':'IsDeletedDeal','Id':'DealId' , 'Id.1':'ActivityId','IsDeleted.1':'IsDeletedActivity','RegisterTime':'RegisterTimeDeal',
                                                                                    'Status':'StatusDeal','RegisterDate':'RegisterDateActivity','IsDone':'IsDoneActivity',
                                                                                    'DoneDate':'DoneDateActivity'})
df['DealId']=df['DealId'].str.lower()
df['DoneDateActivity'] = np.where(df['IsDoneActivity']==False , np.nan ,df['DoneDateActivity'])
df['DealFinishTime']=np.where(df['StatusDeal']==1 , df['ChangeToWonTime'] , 
                                np.where(df['StatusDeal']==2 , df['ChangeToLossTime'] , None))
indecies=[]
def ApplyCoerceError(row):
    try:
        y=pd.to_datetime(row)
        return True
    except Exception:
        return False
indecies = df[['RegisterDateActivity' , 'DoneDateActivity' , 'DealFinishTime' , 'RegisterTimeDeal']].apply(ApplyCoerceError, axis=1).tolist()
shape1=df.shape[0]
print('''Records of corrupted data registered by BizDomains are saved 
in a csv file named "corrupted_Deals.csv" and you can check the times for each''')
df[~np.array(indecies)].to_csv('corrupted_Deals.csv')
df = df[indecies]
shape2=df.shape[0]
print(f'number of problematic cases in datetime entrance: {shape1-shape2}, number of rows before: {shape1}, number of rows after: {shape2}')

df['RegisterDateActivity'] = pd.to_datetime(df['RegisterDateActivity'])
df['DoneDateActivity'] = pd.to_datetime(df['DoneDateActivity'] )
df['RegisterTimeDeal'] = pd.to_datetime(df['RegisterTimeDeal'])
df['DealFinishTime'] = pd.to_datetime(df['DealFinishTime'])

# Deleting the rows with unconventional type of date/time which was stored as NaT
df.drop(['ChangeToWonTime' , 'ChangeToLossTime' , 'ActivityId'],axis=1,inplace=True)
df.head()

Records of corrupted data registered by BizDomains are saved 
in a csv file named "corrupted_Deals.csv" and you can check the times for each
number of problematic cases in datetime entrance: 3311, number of rows before: 2152632, number of rows after: 2149321


,DealId,IsDeletedDeal,RegisterTimeDeal,StatusDeal,IsDeletedActivity,RegisterDateActivity,IsDoneActivity,DoneDateActivity,DealFinishTime
0,cef52513-8f12-4601-91f2-00002ce80633,False,2022-04-14 21:19:41.000,1,NaN,NaT,NaN,NaT,2022-04-14 21:20:16.480
1,5977d067-a390-4b9c-9d58-00003b1f11cf,False,2022-06-08 12:37:08.210,0,NaN,NaT,NaN,NaT,NaT
2,80f17f47-87a7-4259-b234-00004c7c6056,False,2022-06-24 19:03:12.280,1,NaN,NaT,NaN,NaT,2022-06-24 19:03:12.280
3,c6ad7211-1856-4129-b3b0-000053ded53b,False,2022-04-10 08:37:00.803,1,NaN,NaT,NaN,NaT,2022-04-10 08:37:00.803
4,30322eb0-a08d-4b98-80e2-000067a52945,False,2022-06-13 08:11:20.647,0,NaN,NaT,NaN,NaT,NaT


# Joining Biz Dataset

If BizDomainIds are included in the initial dataset, just apply line 7 and 8 of the code cell below on the given df

In [3]:
biz1=pd.read_csv(r"Bizdomain1.csv")
biz2=pd.read_csv(r"Bizdomain2.csv")

# complete bizdomains (appending 2 sheets of Bizdomain excel)
biz=pd.concat([biz1,biz2])
print(biz.shape)
biz['DealId']=biz['DealId'].str.lower()
biz['BizDomainId']=biz['BizDomainId'].str.lower()

#test for merging of complete biz and 100 rows of file01 which was calcultaed for interval calculs
df2 = pd.merge(df, biz, how='left')
df2.head()

(1122804, 2)


,DealId,IsDeletedDeal,RegisterTimeDeal,StatusDeal,IsDeletedActivity,RegisterDateActivity,IsDoneActivity,DoneDateActivity,DealFinishTime,BizDomainId
0,cef52513-8f12-4601-91f2-00002ce80633,False,2022-04-14 21:19:41.000,1,NaN,NaT,NaN,NaT,2022-04-14 21:20:16.480,314a88a8-6d18-4972-abf5-d701f54dee15
1,5977d067-a390-4b9c-9d58-00003b1f11cf,False,2022-06-08 12:37:08.210,0,NaN,NaT,NaN,NaT,NaT,ca7dd0be-49a6-4b2e-ab21-3616d57e9571
2,80f17f47-87a7-4259-b234-00004c7c6056,False,2022-06-24 19:03:12.280,1,NaN,NaT,NaN,NaT,2022-06-24 19:03:12.280,729423ab-c61c-4b98-8b49-56ac0349e7c9
3,c6ad7211-1856-4129-b3b0-000053ded53b,False,2022-04-10 08:37:00.803,1,NaN,NaT,NaN,NaT,2022-04-10 08:37:00.803,9963048e-7339-4014-8e82-6115c8e337f4
4,30322eb0-a08d-4b98-80e2-000067a52945,False,2022-06-13 08:11:20.647,0,NaN,NaT,NaN,NaT,NaT,986cdeeb-a977-4ffa-ade8-05fc2f566d19


# Joining ExpireDate Dataset

In [4]:
ExpireDate=pd.read_csv(r"ExpireDate.csv").rename(columns={'تاریخ ثبت نام':'BizRegisterDate','تاریخ انقضا':'BizExpireDate' , 'اخرین تاریخ دسترسی':'BizAccessDate',
                                                                                    'یوزر خریداری شده':'BizBoughtUser','آیا فعال است':'BizActive?'})
ExpireDate['BizDomainId']=ExpireDate['BizDomainId'].str.lower()
ExpireDate.head()

,BizDomainId,BizRegisterDate,BizExpireDate,BizAccessDate,BizBoughtUser,BizActive?
0,98de7ea4-44c2-4913-8214-00010aedefe9,10/25/2019,5/16/2021,9/28/2021,15,0
1,b00f0ace-2581-437b-9ab4-00975f3d3fc0,10/10/2021,10/25/2022,4/30/2022,1,0
2,69a4c374-98b3-400d-aeae-00b4db07c67c,1/2/2021,2/8/2021,5/25/2021,2,0
3,0ae71d29-e241-4fb1-890e-01000311b28d,4/21/2016,8/10/2016,2/9/2017,5,0
4,c53d80e3-b31c-43aa-9451-02b97be31677,1/2/2020,5/4/2021,5/9/2022,30,0


Pay attention to `exportdate` as it should be changed for preprocessing th train and test data.

In [5]:
df2 = pd.merge(df2, ExpireDate, how='left')
df2['BizRegisterDate'] = pd.to_datetime(df2['BizRegisterDate'])
df2['BizExpireDate'] = pd.to_datetime(df2['BizExpireDate'])
df2['BizAccessDate'] = pd.to_datetime(df2['BizAccessDate'])
exportdate=pd.Timestamp('2022-7-31T12') # insert your export date
df2['BizIsExpired?']=np.where(df2['BizExpireDate']>=exportdate,0,1)
df2.head()

,DealId,IsDeletedDeal,RegisterTimeDeal,StatusDeal,IsDeletedActivity,RegisterDateActivity,IsDoneActivity,DoneDateActivity,DealFinishTime,BizDomainId,BizRegisterDate,BizExpireDate,BizAccessDate,BizBoughtUser,BizActive?,BizIsExpired?
0,cef52513-8f12-4601-91f2-00002ce80633,False,2022-04-14 21:19:41.000,1,NaN,NaT,NaN,NaT,2022-04-14 21:20:16.480,314a88a8-6d18-4972-abf5-d701f54dee15,2022-01-12,2023-01-27,2022-07-30,3.0,1.0,0
1,5977d067-a390-4b9c-9d58-00003b1f11cf,False,2022-06-08 12:37:08.210,0,NaN,NaT,NaN,NaT,NaT,ca7dd0be-49a6-4b2e-ab21-3616d57e9571,2019-07-18,2022-09-26,2022-07-31,3.0,1.0,0
2,80f17f47-87a7-4259-b234-00004c7c6056,False,2022-06-24 19:03:12.280,1,NaN,NaT,NaN,NaT,2022-06-24 19:03:12.280,729423ab-c61c-4b98-8b49-56ac0349e7c9,2020-08-10,2022-11-15,2022-07-31,5.0,1.0,0
3,c6ad7211-1856-4129-b3b0-000053ded53b,False,2022-04-10 08:37:00.803,1,NaN,NaT,NaN,NaT,2022-04-10 08:37:00.803,9963048e-7339-4014-8e82-6115c8e337f4,2021-09-24,2022-11-02,2022-07-31,4.0,1.0,0
4,30322eb0-a08d-4b98-80e2-000067a52945,False,2022-06-13 08:11:20.647,0,NaN,NaT,NaN,NaT,NaT,986cdeeb-a977-4ffa-ade8-05fc2f566d19,2021-10-16,2022-10-31,2022-07-31,1.0,1.0,0


#### Changing all T/F to 1/0

In [6]:
df2['IsDeletedDeal']=df2['IsDeletedDeal'].map({True: 1, False: 0})
df2['IsDeletedActivity']=df2['IsDeletedActivity'].map({True: 1, False: 0})
df2['IsDoneActivity']=df2['IsDoneActivity'].map({True: 1, False: 0})
df2.head()

,DealId,IsDeletedDeal,RegisterTimeDeal,StatusDeal,IsDeletedActivity,RegisterDateActivity,IsDoneActivity,DoneDateActivity,DealFinishTime,BizDomainId,BizRegisterDate,BizExpireDate,BizAccessDate,BizBoughtUser,BizActive?,BizIsExpired?
0,cef52513-8f12-4601-91f2-00002ce80633,0,2022-04-14 21:19:41.000,1,NaN,NaT,NaN,NaT,2022-04-14 21:20:16.480,314a88a8-6d18-4972-abf5-d701f54dee15,2022-01-12,2023-01-27,2022-07-30,3.0,1.0,0
1,5977d067-a390-4b9c-9d58-00003b1f11cf,0,2022-06-08 12:37:08.210,0,NaN,NaT,NaN,NaT,NaT,ca7dd0be-49a6-4b2e-ab21-3616d57e9571,2019-07-18,2022-09-26,2022-07-31,3.0,1.0,0
2,80f17f47-87a7-4259-b234-00004c7c6056,0,2022-06-24 19:03:12.280,1,NaN,NaT,NaN,NaT,2022-06-24 19:03:12.280,729423ab-c61c-4b98-8b49-56ac0349e7c9,2020-08-10,2022-11-15,2022-07-31,5.0,1.0,0
3,c6ad7211-1856-4129-b3b0-000053ded53b,0,2022-04-10 08:37:00.803,1,NaN,NaT,NaN,NaT,2022-04-10 08:37:00.803,9963048e-7339-4014-8e82-6115c8e337f4,2021-09-24,2022-11-02,2022-07-31,4.0,1.0,0
4,30322eb0-a08d-4b98-80e2-000067a52945,0,2022-06-13 08:11:20.647,0,NaN,NaT,NaN,NaT,NaT,986cdeeb-a977-4ffa-ade8-05fc2f566d19,2021-10-16,2022-10-31,2022-07-31,1.0,1.0,0


## Deleting data if IsDeletedDeal=1 , StatusDeal=0

In [7]:
indecies = df2[(df2['IsDeletedDeal']==1) & (df2['StatusDeal']==0)].index
df2 = df2.drop(indecies)
df2.shape

(1970974, 16)

## Function for Deals and Activities for Fixing the Finish Time

In [8]:
def ApplyIterFunction(row):
    '''
    input:
    each row of the dataframe
    
    ----------------------
    
    output:
    This changes are made:
    
    - if a Deal is "ongoing" and "not deleted", and its Bizdomain is "expired",deal's finish time 
      equals to minimum of access date and expire date. Moreover, if it has a activity which is "ongoing"
      and "not deleted", the activity's finish time also equals to minimum of access date and expire date. 
      If the deal is ongoing for real (meaning that the bizmoain is not expired) the finish time of the deal 
      equals to the date when we exported the data. Additionally, f it has a activity which is "ongoing"
      and "not deleted", the activity's finish time also equals to export date.
      
    - if the Deal is "Done" before and "not deleted", AND its "activity is not deleted", 
      the done date of the activity equals to the related deal finish time.
      
    - If a deal has an activity, the register time of the activity shoud equal to minimum of 
      register time of the activity AND finish time of that.
    '''
    
    if (row['StatusDeal']==0) and (row['IsDeletedDeal']==0):
        if row['BizIsExpired?']==1:
            #calculating the last time user accessed to didar (which is the min time of accessed date and expire date)
            dummy=min(row['BizExpireDate'] , row['BizAccessDate'])
            row['DealFinishTime']=dummy    # FinishTime of a expired bizdomain which has an ongoing deal
            
            # checking the activity
            if row['IsDoneActivity']==0 and row['IsDeletedActivity']==0:
                row['DoneDateActivity']=dummy
        
        # the deal is ongoing for real
        else:
            row['DealFinishTime']=exportdate
            if row['IsDoneActivity']==0 and row['IsDeletedActivity']==0:
                row['DoneDateActivity']=exportdate
                
    # replace the time of DoneDateActivity with DealFinishTime for Activities that arent complete and not deleted
    # also their deal is okay (not deleted and completed)
    elif (row['StatusDeal']!=0) and (row['IsDeletedDeal']==0):
        if row['IsDeletedActivity']==0:
            row['DoneDateActivity']=min(row['DealFinishTime'] , row['DoneDateActivity'])
            
    # fixing the register time of the OKAY activities if RegTime>DoneDate      
    if not pd.isna(row['RegisterDateActivity']):
        row['RegisterDateActivity'] = min(row['RegisterDateActivity'],row['DoneDateActivity'])
        
    return row

In [9]:
df2 = df2.apply(ApplyIterFunction, axis=1)
df2.head()

,DealId,IsDeletedDeal,RegisterTimeDeal,StatusDeal,IsDeletedActivity,RegisterDateActivity,IsDoneActivity,DoneDateActivity,DealFinishTime,BizDomainId,BizRegisterDate,BizExpireDate,BizAccessDate,BizBoughtUser,BizActive?,BizIsExpired?
0,cef52513-8f12-4601-91f2-00002ce80633,0,2022-04-14 21:19:41.000,1,NaN,NaT,NaN,NaT,2022-04-14 21:20:16.480,314a88a8-6d18-4972-abf5-d701f54dee15,2022-01-12,2023-01-27,2022-07-30,3.0,1.0,0
1,5977d067-a390-4b9c-9d58-00003b1f11cf,0,2022-06-08 12:37:08.210,0,NaN,NaT,NaN,NaT,2022-07-31 12:00:00.000,ca7dd0be-49a6-4b2e-ab21-3616d57e9571,2019-07-18,2022-09-26,2022-07-31,3.0,1.0,0
2,80f17f47-87a7-4259-b234-00004c7c6056,0,2022-06-24 19:03:12.280,1,NaN,NaT,NaN,NaT,2022-06-24 19:03:12.280,729423ab-c61c-4b98-8b49-56ac0349e7c9,2020-08-10,2022-11-15,2022-07-31,5.0,1.0,0
3,c6ad7211-1856-4129-b3b0-000053ded53b,0,2022-04-10 08:37:00.803,1,NaN,NaT,NaN,NaT,2022-04-10 08:37:00.803,9963048e-7339-4014-8e82-6115c8e337f4,2021-09-24,2022-11-02,2022-07-31,4.0,1.0,0
4,30322eb0-a08d-4b98-80e2-000067a52945,0,2022-06-13 08:11:20.647,0,NaN,NaT,NaN,NaT,2022-07-31 12:00:00.000,986cdeeb-a977-4ffa-ade8-05fc2f566d19,2021-10-16,2022-10-31,2022-07-31,1.0,1.0,0


## Activity: IsDone==0 & IsDeleted==1

In [10]:
mask = (df2['IsDoneActivity'] == 0) & (df2['IsDeletedActivity'] == 1)
u_biz_id = list(set(df2.loc[mask,'BizDomainId']))

#### Finding the correct activities of the bizdomains that have problems (in roder to reach out to their means)

In [11]:
# Use this line to get rid of the replacement warning
pd.options.mode.chained_assignment = None
res = []
first_part_biz = []
for biz in u_biz_id:
    mask = (df2['BizDomainId'] == biz)  & (df2['IsDoneActivity'] == 1)
    dif = (df2.loc[mask,'DoneDateActivity'] - df2.loc[mask,'RegisterDateActivity']).dt.total_seconds()
    # Some of the activities' duration are showed to be negative, we replace them with zero
    dif[dif<0] = 0
    # Compute the average interval time, in hours unit
    mean = np.mean(dif)/3600
    if not (pd.isna(mean)):
        res.append((biz,mean))
        first_part_biz.append(biz)
# res

#### BizDomanis without any correct activities to calculate their means

In [12]:
# we must find other bizdomains mean
# Generating unique list of biz domains
u_bizset = set(df2['BizDomainId'])
# Computing the difference between u_bizset and the problematic bizdomain set
final_biz = u_bizset.difference(set(u_biz_id))
# Computing mean for each bizid in the final_biz list
temp = []
for biz in final_biz:
    mask = (df2['BizDomainId'] == biz) & (df2['IsDoneActivity'] == 1)
    dif = (df2.loc[mask,'DoneDateActivity'] - df2.loc[mask,'RegisterDateActivity']).dt.total_seconds() 
    # Some of the activities' duration are showed to be negative, we replace them with zero
    dif[dif<0] = 0
    # Compute the average interval time, in hours unit
    mean = np.mean(dif)/3600
    if not (pd.isna(mean)):
        temp.append((biz,mean))
  # Computing the global mean
temp.extend(res)
g_mean = sum(elt[1] for elt in temp)/len(temp)
second_part_biz = list(set(u_biz_id).difference(first_part_biz))

for biz in second_part_biz:
    res.append((biz,g_mean))
    
for el in res:
    mask = (df2['BizDomainId']==el[0]) & (df2['IsDoneActivity'] == False) & (df2['IsDeletedActivity'] == True)
    df2.loc[mask,'DoneDateActivity'] = df2.loc[mask,'RegisterDateActivity'] + pd.Timedelta(el[1],"hours")

## Function for Fixing the order of the times of occurrence
#### DealStartTime<ActivityStartTime<ActivityFinishTime<DealFinishTime

In [13]:
def ApplyFixingTime(row):
    
    row['RegisterTimeDeal']=min(row['RegisterTimeDeal'] , row['DealFinishTime'])
    if not pd.isna(row['RegisterDateActivity']):
        row['RegisterDateActivity'] = max(row['RegisterDateActivity'] , row['RegisterTimeDeal'])
        row['DoneDateActivity'] = min(row['DoneDateActivity'] , row['DealFinishTime'])
        row['DoneDateActivity'] = max(row['DoneDateActivity'] , row['RegisterDateActivity'])
    return row

In [14]:
df2 = df2.apply(ApplyFixingTime , axis=1)

# Exporting the cleaned dataset

Export data based on the shape of the df, each 1,000,000 rows should be saved a separate csv file

In [15]:
df2.shape

(1970974, 16)

In [20]:
os.mkdir('files_cleaned')
df2.loc[:1000000,:].to_csv('files_cleaned\df_file_cleaned1.csv')
df2.loc[1000000:,:].to_csv('files_cleaned\df_file_cleaned2.csv')

# Importing the cleaned dataset

Change the directory of exported data to `files_cleaned` and then use the cell below if you desire to use them. Exporting should be done because the cleaned dataset will be used in future Feature Extractions.

In [21]:
path = r"files_cleaned" # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

df2 = pd.concat(li, axis=0, ignore_index=True)
df2.loc[:, ['RegisterTimeDeal' , 'DealFinishTime','RegisterDateActivity','DoneDateActivity']] = df2.loc[:,['RegisterTimeDeal' , 'DealFinishTime','RegisterDateActivity','DoneDateActivity']].apply(pd.to_datetime)
df2.head()

,DealId,IsDeletedDeal,RegisterTimeDeal,StatusDeal,IsDeletedActivity,RegisterDateActivity,IsDoneActivity,DoneDateActivity,DealFinishTime,BizDomainId,BizRegisterDate,BizExpireDate,BizAccessDate,BizBoughtUser,BizActive?,BizIsExpired?
0,cef52513-8f12-4601-91f2-00002ce80633,0,2022-04-14 21:19:41.000,1,NaN,NaT,NaN,NaT,2022-04-14 21:20:16.480,314a88a8-6d18-4972-abf5-d701f54dee15,2022-01-12,2023-01-27,2022-07-30,3.0,1.0,0
1,5977d067-a390-4b9c-9d58-00003b1f11cf,0,2022-06-08 12:37:08.210,0,NaN,NaT,NaN,NaT,2022-07-31 12:00:00.000,ca7dd0be-49a6-4b2e-ab21-3616d57e9571,2019-07-18,2022-09-26,2022-07-31,3.0,1.0,0
2,80f17f47-87a7-4259-b234-00004c7c6056,0,2022-06-24 19:03:12.280,1,NaN,NaT,NaN,NaT,2022-06-24 19:03:12.280,729423ab-c61c-4b98-8b49-56ac0349e7c9,2020-08-10,2022-11-15,2022-07-31,5.0,1.0,0
3,c6ad7211-1856-4129-b3b0-000053ded53b,0,2022-04-10 08:37:00.803,1,NaN,NaT,NaN,NaT,2022-04-10 08:37:00.803,9963048e-7339-4014-8e82-6115c8e337f4,2021-09-24,2022-11-02,2022-07-31,4.0,1.0,0
4,30322eb0-a08d-4b98-80e2-000067a52945,0,2022-06-13 08:11:20.647,0,NaN,NaT,NaN,NaT,2022-07-31 12:00:00.000,986cdeeb-a977-4ffa-ade8-05fc2f566d19,2021-10-16,2022-10-31,2022-07-31,1.0,1.0,0


# Function for calculating the metric of proportion of warning time to total deal time

#### calculate the merge time

In [22]:
def merge_times(intervals):
    intervals.sort(key = lambda x: x[0])
    
    res = [intervals[0]]
    for idx in range(1,len(intervals)):
        curr = intervals[idx]
        prev = res[-1]
        
        # Check for overlap
        if prev[1] < curr[0]: # No overlap
            res.append(curr)
        else:
            # Overlap, update end time
            prev[1] = max(prev[1], curr[1])
        
    return res

#### calculate the merge ratio

In [23]:
def no_warning_ratio(start_time , end_time , intervals):
    '''
    input:
    start_time = RegisterTime of a deal
    end_time = DealFinishTime
    intervals = related intervals for activities on a specific deal
    
    ----------------------
    
    output:
    ID of the activity
    ratio of (not showing warning signal)/(duration of the deal)
    ** The closer the output is to 1, the better the BIZdomain is performing **
    '''
    end_time_int = pd.Timedelta(end_time-start_time).total_seconds()

    if pd.isna(intervals.values[0][0]):
        return [0,end_time_int]
    elif end_time_int ==0 :
        return [0,0]
    else:
        intervals = [list(y.total_seconds() for y in row) for _,row in (intervals - start_time).iterrows()]
        merged = merge_times(intervals)
        with_activity = sum([(x[1]-x[0]) for x in merged])
        ratio = with_activity/end_time_int
        return [ratio,end_time_int]

#### Work as an input function for groupby

In [24]:
def metric_activation_function(df):
    '''
    input:
    dataframe of the groupby function applied to DealIds
    
    ----------------------
    
    output:
    - BizId
    - DealId
    - ratio fo no warning sign
    '''
    
    BizId = df.loc[:,'BizDomainId'].values[0]
    DealId = df.loc[: , 'DealId'].values[0]
    start_time = df.loc[:,'RegisterTimeDeal'].values[0]
    end_time = df.loc[:,'DealFinishTime'].values[0]
    intervals = df.loc[:,['RegisterDateActivity' ,'DoneDateActivity']]
    res = no_warning_ratio(start_time , end_time , intervals)
    return {'BizDomainId':BizId , 'DealId':DealId ,'metric':res[0] , 'DealDuration':res[1]}

In [25]:
metric = df2.groupby('DealId').apply(metric_activation_function)
metric = pd.DataFrame(metric.tolist())

## Aggregating for BizId by the means of Calculating expectation (weighted average)

In [26]:
def AggreagtorBizId(df):
    BizId=df.loc[:,'BizDomainId'].values[0]
    if np.sum(df['DealDuration'])==0:
        return {'BizDomainId':BizId , 'no_warning_sign':0}
    else:
        return {'BizDomainId': BizId , 'no_warning_sign':
               np.sum(df['metric']*df['DealDuration'])/df['DealDuration'].sum()}

In [27]:
final_df = metric.groupby('BizDomainId').apply(AggreagtorBizId)
final_df = pd.DataFrame(final_df.tolist())

# Merging Expired or Not and Active or not with final df

In [28]:
final_df = pd.merge(final_df , df2[['BizDomainId','BizIsExpired?','BizActive?']] , how='left')
final_df.head()

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?
0,0006df37-709c-4121-be9e-5210adc6a044,0.0,1,0.0
1,0006df37-709c-4121-be9e-5210adc6a044,0.0,1,0.0
2,0006df37-709c-4121-be9e-5210adc6a044,0.0,1,0.0
3,0006df37-709c-4121-be9e-5210adc6a044,0.0,1,0.0
4,0006df37-709c-4121-be9e-5210adc6a044,0.0,1,0.0


# Output

In [29]:
final_df = final_df.drop_duplicates()
final_df.to_csv('df_train.csv')